In [1]:
!pip install chromadb
!pip install -U duckduckgo-search
!pip install pdfkit
!pip install -U langchain-community
!pip install -U langchain-google-genai
%pip install --upgrade --quiet duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 2.2 MB/s eta 0:00:00


In [2]:
!pip install -q streamlit
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
# import subprocess
# subprocess.Popen(["./cloudflared-linux-amd64", "tunnel", "--url", "http://localhost:8501"])
!nohup /content/cloudflared-linux-amd64 tunnel --url http://localhost:8501 &

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.8 MB/s eta 0:00:00
--2024-12-08 18:12:06--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2024.11.1/cloudflared-linux-amd64 [following]
--2024-12-08 18:12:07--  https://github.com/cloudflare/cloudflared/releases/download/2024.11.1/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-rele

In [5]:
!grep -o 'https://.*\.trycloudflare.com' nohup.out | head -n 1 | xargs -I {} echo "Your tunnel url {}"

Your tunnel url https://correspondence-giving-appear-represent.trycloudflare.com


In [4]:
!streamlit run /content/app.py &>/content/logs.txt &

In [3]:
%%writefile app.py
import streamlit as st

import os
import pdfkit
import pathlib
import textwrap
import requests
import streamlit as st
from io import BytesIO
import base64
from bs4 import BeautifulSoup
import google.generativeai as genai
from IPython.display import display, Markdown
import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
warnings.filterwarnings("ignore")

os.environ['GOOGLE_API_KEY'] = 'Enter_your_API_Key'

from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import re
import time

def get_links(keywords):
    wrapper = DuckDuckGoSearchAPIWrapper(max_results=3)
    search = DuckDuckGoSearchResults(api_wrapper=wrapper)
    time.sleep(2)  # Pause for 2 seconds before making the request
    results = search.run(tool_input=keywords)

    links = []
    parsed_links = re.findall(r'link:\s*(https?://[^\],\s]+)', results)

    for link in parsed_links:
        links.append(link)

    return links

from langchain_community.document_loaders import WebBaseLoader
import bs4
from langchain_text_splitters import RecursiveCharacterTextSplitter

def scrape_content(keywords):
  bs4_strainer = bs4.SoupStrainer(('p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'))
  time.sleep(2)  # Pause for 2 seconds before making the request
  document_loader = WebBaseLoader(web_path=(get_links(keywords)))
  docs = document_loader.load()

  splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200,add_start_index=True,)
  splits = splitter.split_documents(docs)

  embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
  vector_store = Chroma.from_documents(documents=splits, embedding=embeddings)
  retriever = vector_store.as_retriever(search_type="similarity", search_kwards={"k": 10})

  return retriever

def to_markdown(text):
    text = text.replace('.', '*')
    return textwrap.indent(text, '>', predicate=lambda _: True)

import pandas as pd

path = 'https://raw.githubusercontent.com/manishk1332/blog_writer/refs/heads/main/grouped_output.json'
df = pd.read_json(path)

def extract_relevant_documents(pdf_path, query):
    pdf_loader = PyPDFLoader(pdf_path)
    pages = pdf_loader.load_and_split()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=0)
    context = "\n\n".join(str(page.page_content) for page in pages)
    texts = text_splitter.split_text(context)

    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vector_index = Chroma.from_texts(texts, embeddings).as_retriever()

    relevant_documents = vector_index.get_relevant_documents(query)

    return relevant_documents

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

def generate_blog(topic, category, audience='all', article_type='general', keywords=None, retriever=None, knowledge=None):

    if audience == "young professionals":
        tone = "conversational and engaging"
    elif audience == "senior citizens":
        tone = "formal and empathetic"
    else:
        tone = "neutral and informative"

    if retriever:
        retrieved_docs = retriever.get_relevant_documents(topic)
        retrieved_knowledge = "\n\n".join([doc.page_content for doc in retrieved_docs])
    else:
        retrieved_knowledge = ""

    combined_knowledge = (knowledge or "") + "\n\n" + retrieved_knowledge

    header_prompt = f"""
    You are a content strategist at Ditto Insurance, a reputed organization which helps users choose the best policy for their needs, writing blogs on {category}.
    Write an SEO-optimized, 14-word blog heading about {topic}.
    Align the heading with Ditto's brand voice: transparent, user-centric, and trustworthy. Incorporate alert words like 'risk' or 'avoid'
    while keeping it engaging for the {audience}.
    """
    heading = llm.invoke(header_prompt).content

    if article_type == 'review':
        link = df[category]['all']['links'][1]
        eg_topic = df[category]['all']['titles'][1]
    elif article_type == 'comparison':
        link = df[category]['all']['links'][2]
        eg_topic = df[category]['all']['titles'][2]
    else:
        link = df[category][audience]['links'][0]
        eg_topic = df[category][audience]['titles'][0]
    res = requests.get(link)

    soup = BeautifulSoup(res.text,'html.parser')
    eg_content = soup.find('div', class_='relative post-content sm:px-6 md:px-0 load-external-scripts gh-content').text

    # Blog content prompt
    blog_prompt = f"""
    You are a senior blog writer for Ditto Insurance where your task is to write SEO optimized and SERP analyzed informational blogs within the category of '{category}'.
    Write an audience-specific blog of 2000 words on the topic '{topic}'.
    The number of words in the blog must not be less than 1500 words. Write a long blog.
    Use a {tone} tone and incorporate examples from Ditto's existing content style. Use the following example of a blog on {eg_topic} to get an idea, with the following content: {eg_content}.
    The style of writing should strictly follow:\n
    {eg_content}\n
    Highlight Ditto's role in simplifying insurance and helping users make informed decisions.
    Emphasize on including and writing the blog around the following keywords: '{keywords}'.
    Structure the blog with headings (h2, h3), avoid bullet points, and weave relevant audience details (e.g., age groups, premium brackets).
    Use the following knowledge to enrich the blog:\n
    {combined_knowledge}
    """
    blog_content = llm.invoke(blog_prompt).content

    blog = f"# {heading}\n\n{blog_content}"
    return blog

def write_blog(pdf_path, prompt, cat):
    docs = extract_relevant_documents(pdf_path, prompt)
    blog = generate_blog(topic=prompt,
           category=cat, knowledge = docs)
    return to_markdown(blog)

st.title('Blog Generator')

input_text = st.text_area("Enter your text here", height=300)
prompt = st.text_input("Enter the blog topic")
category = st.selectbox("Choose the blog category", ['Health Insurance', 'Life Insurance'])
audience = st.selectbox("Choose the audience", ['young professionals', 'senior citizens', 'all'])
article_type = st.selectbox("Choose the article type", ['general', 'review', 'comparison'])
keywords = st.text_input("Enter the keywords")

if st.button('Generate Blog'):
    if prompt:
        retriever = scrape_content(prompt)
        blog_response = generate_blog(prompt, category, audience, article_type, keywords, retriever, input_text)
        st.markdown(blog_response, unsafe_allow_html=True)
    else:
        st.error("Please enter a prompt.")

Writing app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.225.229:8501

⠴⠦Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦your url is: https://eighty-cougars-nail.loca.lt
/usr/local/lib/python3.10/dist-packages/langchain/__init__.py:30: UserWarning: Importing PromptTemplate from langchain root module is no longer supported. Please use langchain_core.prompts.PromptTemplate instead.
  warnings.warn(
/content/app.py:22: LangChainDeprecationWarning: Importing PyPDFLoader from langchain.document_loaders is deprecated. Please replace deprecated imports:

>> from langchain.document_loaders import PyPDFLoader

with new imports of:

>> from langchain_community.document_loaders import PyPDFLoader
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/

In [ ]:
path = 'https://raw.githubusercontent.com/manishk1332/blog_writer/refs/heads/main/grouped_output.json'
df = pd.read_json(path, lines=True)

In [ ]:
df

,Health Insurance,Life Insurance
all,"{'blogs': [None, None, None], 'links': ['https...","{'blogs': [None, None, None], 'links': ['https..."
senior citizens,"{'blogs': [None], 'links': ['https://joinditto...","{'blogs': [None], 'links': ['https://joinditto..."
young professionals,"{'blogs': [None, None, None], 'links': ['https...","{'blogs': [None, None, None], 'links': ['https..."
